In [2]:
import tweepy
import json
import datetime
import pandas as pd

# set up API

In [3]:
consumer_key='uKZylT3WsDp6nnDUy4QjR9LQA'
consumer_secret='9rJT6UiZGni4JCSBkXOEsbgkNhqiufMj6eACcNyjGapNA0TdH8'
access_token='1098355138165698560-fHvSVTMajUk8IzPhhQs7Ee0LrveGBe'
access_token_secret='DLqslWOlsurmFTX3uYL6f3Sm0DBjaI8ByzcP4KwyBYCmM'
            
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

collect 

In [4]:
#open file
filename = "movie_news.csv"
f = open(filename,"w")

#write header in csv
headers = "news_id,user_name,screen_name,hashtags,time_of_tweet,retweet_count,news_tweet\n"
f.write(headers)

#this is all the movie related account, so I can 99% sure that my news is related to news not something else
usersets = ['Variety','THR','EntInsider','AMCTheatres','DRMovieNews1','Fandango','RottenTomatoes','animationaddict','cineworld','UniversalPicsAU','DWAnimation','EW','IMDb','20thcenturyfox','movies_anywhere','Marcus_Theatres','WarnerBrosEnt','Lionsgate']

for userset in usersets: #for every user

    user = api.get_user(userset) #get all user info
    
    #get the user's tweets in 24 hours
    
    tweets_24h_list = []
    
    def get_tweets(api, username):
        page = 1
        deadend = False
        while not deadend:
            tweets = api.user_timeline(userset, page = page)
            for tweet in tweets:
                if (datetime.datetime.now() - tweet.created_at).days < 1: #in 1 day=in 24 hours
                    tweets_24h_list.append(tweet.text)
                else:
                    deadend = True
                    return
            
            if not deadend:
                page+=1

    #collect all other info I want
    get_tweets(api, userset)
    tweets_24h_num = len(tweets_24h_list)
    new_tweets = api.user_timeline(userset,count=100, tweet_mode="extended")
    
    for new_tweet in new_tweets:
        json_tweet = new_tweet._json
        json_str = json.dumps(new_tweet._json)
        json_dict = json.loads(json_str)
        main_user_dataset = json_dict['user']
        screen_name = main_user_dataset['screen_name']
        user_name = main_user_dataset['name']
        followers_num_int = main_user_dataset['followers_count']
        followers_num = str(followers_num_int)
        number_of_tweets_in_24_int=tweets_24h_num
        number_of_tweets_in_24 = str(number_of_tweets_in_24_int)
        tweet = json_dict['full_text'].replace(",","|")
        time_of_tweet = str(json_dict['created_at'])
        friends_count_int = main_user_dataset['friends_count']
        friends_count = str(friends_count_int)  
        favorite_count_int = json_dict['favorite_count']
        favorite_count = str(favorite_count_int)
        retweet_count_int = json_dict['retweet_count']
        retweet_count = str(retweet_count_int)
        description= main_user_dataset['description']
        entities = json_dict['entities']
        hashtags_list = entities['hashtags']
        tweet_id = json_dict['id_str']
        if len(hashtags_list) == 0: #some tweet have hashtag some don't, write N/a in those who don't have hashtag
            hashtags_str = 'N/A'
        if len(hashtags_list) != 0: #for those who have hashtags, write all hashtags in a string
            taglist=''
            for x in hashtags_list:
                taglist+='#'
                taglist+=x['text']
                hashtags_str=taglist
        hashtags= hashtags_str
        #write all the info in csv file
        f.write(tweet_id + "," + user_name + "," + screen_name + "," + hashtags.replace(","," ") + "," + time_of_tweet+ "," + retweet_count + "," + tweet.replace("\n","|").replace('"'," ") + " \n")
    
f.close() #close csv file



In [5]:
df = pd.read_csv('movie_news.csv', header=0,low_memory=False)

In [6]:
df.head(15)

,news_id,user_name,screen_name,hashtags,time_of_tweet,retweet_count,news_tweet
0,1121481071042760704,Variety,Variety,NaN,Thu Apr 25 18:28:18 +0000 2019,8,Scarlett Johansson shares her thoughts on the ...
1,1121480244131708933,Variety,Variety,NaN,Thu Apr 25 18:25:00 +0000 2019,1,Indie Sales Acquires Remi Chayé’s Female-Drive...
2,1121478322171666432,Variety,Variety,NaN,Thu Apr 25 18:17:22 +0000 2019,12,Scarlett Johansson on Running for Office: ‘May...
3,1121477413819568128,Variety,Variety,NaN,Thu Apr 25 18:13:46 +0000 2019,10,Joe Biden lines up extensive list of Obama bac...
4,1121477285213822976,Variety,Variety,NaN,Thu Apr 25 18:13:15 +0000 2019,2,New Owner of Gizmodo Media Sites Doesn’t Plan ...
5,1121474987142070272,Variety,Variety,NaN,Thu Apr 25 18:04:07 +0000 2019,5,Netflix acquires Tribeca doc Circus of Books|...
6,1121472825288564736,Variety,Variety,NaN,Thu Apr 25 17:55:32 +0000 2019,1,eOne Names Bill Wilson Senior VP https://t.co/...
7,1121470463950110720,Variety,Variety,NaN,Thu Apr 25 17:46:09 +0000 2019,0,‘EVE Online’ Dev Clears Former GOP Election Of...
8,1121460559545798657,Variety,Variety,#Wahlburgers,Thu Apr 25 17:06:47 +0000 2019,6,#Wahlburgers renewed for 10th and final season...
9,1121455204136886272,Variety,Variety,NaN,Thu Apr 25 16:45:30 +0000 2019,5,From ‘Game of Thrones’ to ‘Big Bang Theory|’ S...


# delete all post without hashtag

In [7]:
df1 = df.dropna(subset=["hashtags"])
df1.head(10)

,news_id,user_name,screen_name,hashtags,time_of_tweet,retweet_count,news_tweet
8,1121460559545798657,Variety,Variety,#Wahlburgers,Thu Apr 25 17:06:47 +0000 2019,6,#Wahlburgers renewed for 10th and final season...
11,1121452307672174596,Variety,Variety,#AvengersEndgame,Thu Apr 25 16:34:00 +0000 2019,16,#AvengersEndgame: How theaters are preparing f...
17,1121444758302408704,Variety,Variety,#Coachella,Thu Apr 25 16:04:00 +0000 2019,2,Justin Bieber on the #Coachella lip-syncing co...
21,1121440982619176960,Variety,Variety,#Bond25,Thu Apr 25 15:49:00 +0000 2019,16,The #Bond25 team revealed plot details and con...
25,1121433685100830720,Variety,Variety,#Empire,Thu Apr 25 15:20:00 +0000 2019,6,Jussie Smollett's character got married on las...
30,1121430057946214402,Variety,Variety,#AvengersEndgame,Thu Apr 25 15:05:35 +0000 2019,23,Josh Brolin is as baffled as we are about *tha...
34,1121426134762004482,Variety,Variety,#AvengersEndgame,Thu Apr 25 14:50:00 +0000 2019,14,Does #AvengersEndgame live up to the hype? Rev...
38,1121422360148680706,Variety,Variety,#AvengersEndgame,Thu Apr 25 14:35:00 +0000 2019,26,#AvengersEndgame hasn't even been released in ...
46,1121410783500312585,Variety,Variety,#Bond25,Thu Apr 25 13:49:00 +0000 2019,276,The #Bond25 filmmakers confirmed that Rami Mal...
70,1121345855749885953,Variety,Variety,#AvengersEndgame,Thu Apr 25 09:31:00 +0000 2019,28,Relive the #AvengersEndgame premiere https://t...


# delete rows when news_tweet is duplicated

In [8]:
df3 = df1.drop_duplicates('news_tweet')

# delete when retweet_count=0

In [9]:
df4 = df3[df3.retweet_count != 0]

# split hashtags by ‘#’

In [10]:
df5=df4.drop('hashtags', axis=1).join(df1['hashtags'].str.split('#', expand=True).stack().reset_index(level=1, drop=True).rename('hashtags'))


In [11]:
clean_z = df5['hashtags'].fillna(0)
clean_z[clean_z==''] = 'N/A'
df5['hashtags'] = clean_z

In [12]:
df6 = df5
df6.head(10)

,news_id,user_name,screen_name,time_of_tweet,retweet_count,news_tweet,hashtags
8,1121460559545798657,Variety,Variety,Thu Apr 25 17:06:47 +0000 2019,6,#Wahlburgers renewed for 10th and final season...,N/A
8,1121460559545798657,Variety,Variety,Thu Apr 25 17:06:47 +0000 2019,6,#Wahlburgers renewed for 10th and final season...,Wahlburgers
11,1121452307672174596,Variety,Variety,Thu Apr 25 16:34:00 +0000 2019,16,#AvengersEndgame: How theaters are preparing f...,N/A
11,1121452307672174596,Variety,Variety,Thu Apr 25 16:34:00 +0000 2019,16,#AvengersEndgame: How theaters are preparing f...,AvengersEndgame
17,1121444758302408704,Variety,Variety,Thu Apr 25 16:04:00 +0000 2019,2,Justin Bieber on the #Coachella lip-syncing co...,N/A
17,1121444758302408704,Variety,Variety,Thu Apr 25 16:04:00 +0000 2019,2,Justin Bieber on the #Coachella lip-syncing co...,Coachella
21,1121440982619176960,Variety,Variety,Thu Apr 25 15:49:00 +0000 2019,16,The #Bond25 team revealed plot details and con...,N/A
21,1121440982619176960,Variety,Variety,Thu Apr 25 15:49:00 +0000 2019,16,The #Bond25 team revealed plot details and con...,Bond25
25,1121433685100830720,Variety,Variety,Thu Apr 25 15:20:00 +0000 2019,6,Jussie Smollett's character got married on las...,N/A
25,1121433685100830720,Variety,Variety,Thu Apr 25 15:20:00 +0000 2019,6,Jussie Smollett's character got married on las...,Empire


In [64]:
df7=df6[df6.hashtags != 'N/A']

In [65]:
df7.head(5)

,news_id,user_name,screen_name,time_of_tweet,retweet_count,news_tweet,hashtags
8,1121460559545798657,Variety,Variety,Thu Apr 25 17:06:47 +0000 2019,6,#Wahlburgers renewed for 10th and final season...,Wahlburgers
11,1121452307672174596,Variety,Variety,Thu Apr 25 16:34:00 +0000 2019,16,#AvengersEndgame: How theaters are preparing f...,AvengersEndgame
17,1121444758302408704,Variety,Variety,Thu Apr 25 16:04:00 +0000 2019,2,Justin Bieber on the #Coachella lip-syncing co...,Coachella
21,1121440982619176960,Variety,Variety,Thu Apr 25 15:49:00 +0000 2019,16,The #Bond25 team revealed plot details and con...,Bond25
25,1121433685100830720,Variety,Variety,Thu Apr 25 15:20:00 +0000 2019,6,Jussie Smollett's character got married on las...,Empire


# delete all tweets start with 'RT @'(only want original tweet)

In [66]:
df8 = df7[~df7.news_tweet.str.startswith('RT @')]

In [78]:
df8.head(5)

,news_id,user_name,screen_name,time_of_tweet,retweet_count,news_tweet,hashtags
8,1121460559545798657,Variety,Variety,Thu Apr 25 17:06:47 +0000 2019,6,#Wahlburgers renewed for 10th and final season...,Wahlburgers
11,1121452307672174596,Variety,Variety,Thu Apr 25 16:34:00 +0000 2019,16,#AvengersEndgame: How theaters are preparing f...,AvengersEndgame
17,1121444758302408704,Variety,Variety,Thu Apr 25 16:04:00 +0000 2019,2,Justin Bieber on the #Coachella lip-syncing co...,Coachella
21,1121440982619176960,Variety,Variety,Thu Apr 25 15:49:00 +0000 2019,16,The #Bond25 team revealed plot details and con...,Bond25
25,1121433685100830720,Variety,Variety,Thu Apr 25 15:20:00 +0000 2019,6,Jussie Smollett's character got married on las...,Empire


In [79]:
df8.reset_index(drop=True, inplace=True)

In [80]:
len(df8)

1018

In [82]:
df9 = df8.drop_duplicates(subset=['news_id','hashtags'])

In [121]:
len(df9)
type(df9)

pandas.core.frame.DataFrame

In [125]:
list_temp_hashtag=df9['hashtags'].to_frame(name='hashtags_sensitive')
type(list_temp_hashtag)
list_temp_hashtag

,hashtags_sensitive
0,Wahlburgers
1,AvengersEndgame
2,Coachella
3,Bond25
4,Empire
5,AvengersEndgame
6,AvengersEndgame
7,AvengersEndgame
8,Bond25
9,AvengersEndgame


In [126]:
df10=pd.concat([df9,list_temp_hashtag],axis=1)

In [128]:
df10
len(df10)

1014

# outport the table 'news_trend'

In [129]:
df10.to_csv('news_trend.csv', sep=',',index=False)

# start to build hashtag table

## take out all the movie hashtags

In [19]:
hashtag1=df8['hashtags'].tolist()

In [20]:
dff = pd.read_csv('new_movie.csv', header=0,low_memory=False)
listt1  = dff['title'].tolist()
listt2 = [x.replace(' ','') for x in listt1]
listt3 = [x.replace(':','') for x in listt2]
listt4 = [x.replace('!','') for x in listt3]
hashtag2 = [x.replace('?','') for x in listt4]
hashtag2

['Penguins',
 'Breakthrough',
 'HailSatan',
 'LittleWoods',
 'HighontheHog',
 'PetSematary',
 'Us',
 'TheBestofEnemies',
 'Unplanned',
 'HowtoTrainYourDragonTheHiddenWorld',
 'TheCurseofLaLlorona',
 'Kalank',
 'UndertheSilverLake',
 'Rafiki',
 'FastColor',
 'Stuck',
 'Shazam',
 'Dumbo',
 'CaptainMarvel',
 'FiveFeetApart',
 'WonderPark']

## write movie title matches its similar hashtags in csv

In [21]:
import difflib
from collections import OrderedDict

In [23]:
filename = "title_hashtags.csv"
f = open(filename,"w")

headers = "movie_title,hashtag\n"
f.write(headers)

for l in listt1:
    h=l.replace(' ','').replace(':','').replace('!','').replace('?','')
    l2=[]
    match1=difflib.get_close_matches(h, hashtag1) #Avenger=avenger=avenger4
    matching = [s for s in hashtag1 if h in s] #AvengerEndgame=Avenger
    match2 = list(OrderedDict.fromkeys(matching))
    match_raw=match1+match2
    match = list(OrderedDict.fromkeys(match_raw)) # delete duplicates
    
    for m in match:
        f.write(l + "," + m + " \n")
    
f.close()





    

# 1. synonyms_hashtag

In [38]:

filename = "title_hashtags_all_synonyms.csv"
f = open(filename,"w")

headers = "hashtag,synonyms_hashtag\n"
f.write(headers)

for l in hashtag1:
    h=l.replace(' ','').replace(':','').replace('!','').replace('?','')
    match1=difflib.get_close_matches(h, hashtag1) #Avenger=avenger=avenger4
    matching = [s for s in hashtag1 if h in s] #AvengerEndgame=Avenger
    match2 = list(OrderedDict.fromkeys(matching))
    match_raw=match1+match2
    match = list(OrderedDict.fromkeys(match_raw)) # delete duplicates
    
    for m in match:
        if m != h:
            f.write(l + "," + m + " \n")
    
f.close()

# 2. spelling mistake or very similar words

In [34]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
#this package can values differences between strings

In [32]:
filename = "title_hashtags_all_spell.csv"
f = open(filename,"w")

headers = "hashtag,misspelling_hashtag\n"
f.write(headers)

for l in hashtag1:
    h=l.replace(' ','').replace(':','').replace('!','').replace('?','')
    l2=[]
    match1=difflib.get_close_matches(h, hashtag1) #Avenger=avenger=avenger4
    matching = [s for s in hashtag1 if h in s] #AvengerEndgame=Avenger
    match2 = list(OrderedDict.fromkeys(matching))
    match_raw=match1+match2
    match = list(OrderedDict.fromkeys(match_raw)) # delete duplicates
    
    for m in match:
        #package fuzz can value the similar rate of 2 string
        #I tried many rate, and found 90% similar rate is apropriate to find spelling mistake
        if fuzz.ratio(m,h)>=90:
            if m != h:
                f.write(l + "," + m + " \n")
    
f.close()

# 3. semantic

### get a movie basic list

In [156]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq

In [243]:
my_url = 'https://www.imdb.com/movies-in-theaters/?ref_=cs_inth'
uClient = uReq( my_url )
page_html = uClient.read()
uClient.close()

In [244]:
page_soup = soup(page_html,"html.parser")
a = page_soup.body.findAll("div",{"class":"list_item even"})
b = page_soup.body.findAll("div",{"class":"list_item odd"})

In [245]:
filename = "new_movie_2.csv"
f = open(filename,"w")

In [246]:
headers = "title\n"
f.write(headers)

6

In [247]:
containers = page_soup.body.findAll("div",{"class":"list_item even"})
for container in containers:
    title = container.tbody.h4.a.text.strip()

    
    f.write(title.replace(","," ") + "\n")
    

In [248]:
containers = page_soup.body.findAll("div",{"class":"list_item odd"})
for container in containers:
    title = container.tbody.h4.a.text.strip()
    
    f.write(title.replace(","," ") + "\n")
 

In [249]:
f.close()

In [250]:
df_movie = pd.read_csv('new_movie_2.csv', header=0,low_memory=False)

In [252]:
#take out all the symbols from movie title, since twitter tag doesn't have any symbols in it
listdf_movie1  = df_movie['title'].tolist()
listdf_movie2 = [x.replace(' ','') for x in listdf_movie1]
listdf_movie3 = [x.replace(':','') for x in listdf_movie2]
listdf_movie4 = [x.replace('!','') for x in listdf_movie3]
df_movie_list = [x.replace('?','') for x in listdf_movie4]
df_movie_list

['TheWhiteCrow',
 'PetSematary',
 'Us',
 'TheBestofEnemies',
 'Unplanned',
 'HowtoTrainYourDragon',
 'Avengers',
 'BeNatural',
 'Shazam',
 'Dumbo',
 'CaptainMarvel',
 'FiveFeetApart',
 'WonderPark',
 'LongShot',
 'UglyDolls',
 'TheIntruder',
 'ElChicano',
 'MeetingGorbachev',
 'Bolden',
 'Clara',
 'PokemonDetectivePikachu',
 'TheHustle',
 'AllIsTrue',
 'Poms',
 'Tolkien',
 'TheBiggestLittleFarm',
 'MySon',
 'TheSilenceofOthers',
 'JohnWick',
 'TheSunIsAlsoaStar',
 'TheSouvenir',
 'TrialbyFire',
 'Aladdin',
 'Booksmart',
 'Brightburn',
 'AdAstra',
 'Halston',
 'Godzilla',
 'Rocketman']

### find tags that are similar to movie titles, write it in csv

In [361]:
filename = "title_hashtags_all_semantic.csv"
f = open(filename,"w")

headers = "category,semantic_hashtag\n"
f.write(headers)



for l in df_movie_list:
    h=l.replace(' ','').replace(':','').replace('!','').replace('?','')
    match1=difflib.get_close_matches(h, hashtag1) 
    # package 'difflib' can get all similar words as a string from a list 
    #Avenger=avenger=avenger4
    matching = [s for s in hashtag1 if h in s] #AvengerEndgame=Avenger
    match2 = list(OrderedDict.fromkeys(matching)) # delete duplicates
    match_raw=match1+match2 #add all the possible matches
    match = list(OrderedDict.fromkeys(match_raw)) # delete duplicates
    
    for m in match:
        f.write('movie_title' + "," + h + " \n")


### find tags that are similar to TV titles, write it in csv

In [362]:
df_tv = pd.read_csv('tv_title.csv', header=0,low_memory=False)
list_tv=df_tv['tv_title'].tolist()

In [363]:
for l in list_tv:
    h=l.replace(' ','').replace(':','').replace('!','').replace('?','')
    match1=difflib.get_close_matches(h, hashtag1) 
    # package 'difflib' can get all similar words as a string from a list 
    matching = [s for s in hashtag1 if h in s] 
    match2 = list(OrderedDict.fromkeys(matching))# delete duplicates
    match_raw=match1+match2
    match = list(OrderedDict.fromkeys(match_raw)) # delete duplicates
    
    for m in match: #write all the match result into a csv file
        f.write('TVseries_title' + "," + h + " \n")


### tags that are not similar to any list I've got

In [364]:
list_all=df_movie_list+list_tv
len(list_all)

70

In [365]:
list_all_add=[]

In [366]:

for l in list_all:
    h=l.replace(' ','').replace(':','').replace('!','').replace('?','')
    match1=difflib.get_close_matches(h, hashtag1) 
    matching = [s for s in hashtag1 if h in s] 
    match2 = list(OrderedDict.fromkeys(matching))
    match_raw=match1+match2
    match = list(OrderedDict.fromkeys(match_raw)) #match is all the similar tags to the list
    for m in match:
        
        if fuzz.ratio(m,h)>=70:
            list_all_add.append(m)

    


In [367]:
len(list_all_add)

36

In [368]:
list_all=list_all+list_all_add

In [369]:
len(list_all)

106

In [370]:
list_all

['TheWhiteCrow',
 'PetSematary',
 'Us',
 'TheBestofEnemies',
 'Unplanned',
 'HowtoTrainYourDragon',
 'Avengers',
 'BeNatural',
 'Shazam',
 'Dumbo',
 'CaptainMarvel',
 'FiveFeetApart',
 'WonderPark',
 'LongShot',
 'UglyDolls',
 'TheIntruder',
 'ElChicano',
 'MeetingGorbachev',
 'Bolden',
 'Clara',
 'PokemonDetectivePikachu',
 'TheHustle',
 'AllIsTrue',
 'Poms',
 'Tolkien',
 'TheBiggestLittleFarm',
 'MySon',
 'TheSilenceofOthers',
 'JohnWick',
 'TheSunIsAlsoaStar',
 'TheSouvenir',
 'TrialbyFire',
 'Aladdin',
 'Booksmart',
 'Brightburn',
 'AdAstra',
 'Halston',
 'Godzilla',
 'Rocketman',
 'GameOfThrones',
 'NewGirl',
 'TheOffice',
 'LionKing',
 'TheBossBaby',
 'ShortyAwards',
 'AmyPoehler',
 'MrsMaisel',
 'EasterWeekend',
 'LadiesNight',
 'FightingWithMyFamily',
 'TopEndWedding',
 'SpiritRidingFree',
 'OnTheBasisOfSex',
 'CannonBusters',
 'Ultraman',
 'TheVillage',
 'InTheDark',
 'UnicornStore',
 'Legacies',
 'AllAmerican',
 'Imposters',
 'WineCountry',
 'PeabodyShow',
 'HappyTogether',
 

In [371]:
for l in list_all:
    for h in hashtag1:
        if l!=h:
            f.write('Other_tags' + "," + h + " \n")

In [372]:
f.close()

### table cleaning

In [373]:
df_semantic = pd.read_csv('title_hashtags_all_semantic.csv', header=0,low_memory=False)

In [374]:
df_semantic2=df_semantic.drop_duplicates()
df_semantic2

,category,semantic_hashtag
0,movie_title,TheWhiteCrow
2,movie_title,Us
6,movie_title,TheBestofEnemies
7,movie_title,HowtoTrainYourDragon
8,movie_title,Avengers
11,movie_title,BeNatural
12,movie_title,Shazam
13,movie_title,CaptainMarvel
15,movie_title,WonderPark
16,movie_title,LongShot


In [398]:
df_semantic2.to_csv('title_hashtags_all_semantic.csv', sep=',',index=False)

## 4. tags in my domain

In [391]:
filename = "title_hashtags_all_domain.csv"
f = open(filename,"w")

headers = "domain,hashtags\n"
f.write(headers)

for l in df_movie_list:
    h=l.replace(' ','').replace(':','').replace('!','').replace('?','')
    match1=difflib.get_close_matches(h, hashtag1) #Avenger=avenger=avenger4
    matching = [s for s in hashtag1 if h in s] #AvengerEndgame=Avenger
    match2 = list(OrderedDict.fromkeys(matching))
    match_raw=match1+match2
    match = list(OrderedDict.fromkeys(match_raw)) # delete duplicates
    
    for m in match:
        f.write('movie' + "," + h + " \n")
f.close()

### cleaning

In [392]:
df_domain = pd.read_csv('title_hashtags_all_domain.csv', header=0,low_memory=False)

In [396]:
df_domain2=df_domain.drop_duplicates()
df_domain2.to_csv('title_hashtags_all_domain.csv', sep=',',index=False)

# Tag tables preview

## 1.Domain tags

In [397]:
df_domain3 = pd.read_csv('title_hashtags_all_domain.csv', header=0,low_memory=False)
df_domain3.head(5)

,domain,hashtags
0,movie,TheWhiteCrow
1,movie,Us
2,movie,TheBestofEnemies
3,movie,HowtoTrainYourDragon
4,movie,Avengers


## 2. mis-spelling tag

In [35]:
df_spell = pd.read_csv('title_hashtags_all_spell.csv', header=0,low_memory=False)
df_spell.head(5)

,hashtag,misspelling_hashtag
0,Deadpool2,Deadpool
1,Mavel,Marvel
2,Deadpool,Deadpool2
3,LitteMovie,LittleMovie
4,BreatkthroughMovie,BreakthroughMovie


## 3. synonyms tag

In [39]:
df_synonyms = pd.read_csv('title_hashtags_all_synonyms.csv', header=0,low_memory=False)
df_synonyms.head(5)

,hashtag,synonyms_hashtag
0,BigLittleLies,LittleMovie
1,WandaVision,DeWandaWise
2,Avengers,AvengersEndgame
3,Avengers,TheAvengers
4,Avengers,AvengersEndgame


## 4. semantic

In [399]:
df_semantic = pd.read_csv('title_hashtags_all_semantic.csv', header=0,low_memory=False)
df_semantic.sample(30)

,category,semantic_hashtag
466,Other_tags,STUBER
237,Other_tags,BlizzardLayoffs
24,movie_title,Godzilla
433,Other_tags,HappyNewYear
164,Other_tags,ad
29,TVseries_title,ShortyAwards
431,Other_tags,TheHateUGive
140,Other_tags,GeminiMan
72,Other_tags,TonyAwards
171,Other_tags,Godzilla
